In [2]:
import pandas as pd

In [3]:
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# Load Environment variables
# ----------------------------------------------------------------------------------------------------------------------
# Load the .env file
load_dotenv()

# Fetch environment variables
db_address = os.getenv('DB_ADDRESS')
db_port = os.getenv('DB_PORT')
db_username = os.getenv('DB_USERNAME')
db_password = quote_plus(os.getenv('DB_PASSWORD'))
db_name = os.getenv('DB')

In [4]:
def get_db_connection():
    """
    Establishes a connection to the PostgreSQL database using SQLAlchemy.
    - The `postgresql+psycopg2` dialect indicates that we're using PostgreSQL as the database backend.
    - The `{DB_USER}:{DB_PASS}` part specifies the username and password to use for connections.
    - The `@{DB_IP}:{DB_PORT}` part specifies the hostname and port number to use when connecting to the database.
    - The `/`${DB}` part specifies the database name or schema to connect to.

    :return: engine.connect(): An active connection object to the PostgreSQL database.
    """
    engine = create_engine(f'postgresql://{db_username}:{db_password}@{db_address}:{db_port}/{db_name}')

    # Establish a connection to the database:
    #   - This returns an active connection object, which can be used to execute queries and interact with the database.
    return engine.connect()

def fetch_data_from_sql(query):
    """
    Fetches data from a SQL table and returns it as a pandas DataFrame.

    :param query: (str) The SQL query to execute on the database.
    :return: pd.DataFrame: A pandas DataFrame containing the results of the SQL query.
    """

    # Establish a connection to the PostgreSQL database using get_db_connection():
    conn = get_db_connection()

    # Execute the SQL query on the database and store the result in a pandas DataFrame:
    df = pd.read_sql(query, conn)

    # Close the connection to the database to free up resources:
    conn.close()

    # Return the pandas DataFrame containing the results of the SQL query:
    return df

def change_column_names(column_list, std=True):
    if std:
        return [col.strip().title().replace('_', ' ') for col in column_list]

    else:
        return [col.strip().lower().replace(' ', '_') for col in column_list]

In [5]:
query_transaction = '''
    SELECT
        *
    FROM
        erp_data.funds_received.transactions
    '''

transaction = fetch_data_from_sql(query_transaction)

In [6]:
 # Get existing master
query_project_dir_old = '''
        SELECT
            *
        FROM
            erp_data.funds_received.project_directory
'''

project_directory = fetch_data_from_sql(query_project_dir_old)

# Standardise column names
project_directory.columns = change_column_names(project_directory.columns)

In [7]:
query_transaction = '''
        SELECT
            *
        FROM
            erp_data.funds_received.wbs
        '''

wbs = fetch_data_from_sql(query_transaction)

In [8]:
# Remove extra spaces from column names
transaction_col = transaction.columns
transaction_col = [col.strip() for col in transaction_col]
transaction.columns = transaction_col

wbs_col = wbs.columns
wbs_col = [col.strip() for col in wbs_col]
wbs.columns = wbs_col

In [9]:
if transaction.shape[0] > 0 and wbs.shape[0] > 0 and project_directory.shape[0] > 0:
    # Perform WBS mapping
    transaction = transaction.merge(wbs[['WBS', 'WBS Details']].drop_duplicates(), how='left',
                                    left_on='Project definition', right_on='WBS')

    transaction = transaction.merge(wbs[['Sub WBS', 'SUB WBS Details']].drop_duplicates(), how='left',
                                    left_on='Object', right_on='Sub WBS')

    transaction['Project Id'] = transaction['SUB WBS Details'].apply(
        lambda x: str(x).strip().lower().replace(' ', '_'))

    # Assign Category
    transaction = transaction.merge(project_directory[['Project Id', 'Category']], how='left', on='Project Id').copy()

In [10]:
start_date = pd.to_datetime('01-04-2024', format='%d-%m-%Y')
end_date = pd.to_datetime('24-12-2024', format='%d-%m-%Y')

In [11]:
# Final Data
final_data = transaction[(transaction['Document Date'].between(start_date, end_date))]

In [12]:
# Starts here
sb_entries = final_data[(final_data['Document type'] == 'SB')].copy()

In [13]:
sb_entries.head()

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category
551,1.002162e+09,2.426005e+09,2024-08-23,2024-08-23,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,510508.0,EXP-SCHOLARSHIP,...,38348.88,38348.88,38348.88,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
552,1.002162e+09,2.426005e+09,2024-08-23,2024-08-23,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,510508.0,EXP-SCHOLARSHIP,...,36109.74,36109.74,36109.74,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
555,1.002165e+09,2.426005e+09,2024-08-28,2024-08-27,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,510508.0,EXP-SCHOLARSHIP,...,38348.88,38348.88,38348.88,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
556,1.002165e+09,2.426005e+09,2024-08-28,2024-08-27,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,510508.0,EXP-SCHOLARSHIP,...,36109.74,36109.74,36109.74,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
912,1.002200e+09,2.426007e+09,2024-10-14,2024-10-01,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,550511.0,Rev-IITBHF Donation,...,-32196387.45,-32196387.45,-32196387.45,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN


In [14]:
final_data.loc[final_data['Document type'] == 'DR', 'Val/COArea Crcy'].sum()

np.float64(-1152370737.54)

In [15]:
project_lists_alloc = [project for project in final_data.loc[final_data['Document type'] == 'DR', 'Object'].drop_duplicates().to_list() if project[-3].isnumeric()]
project_lists_non_alloc = [project for project in final_data.loc[final_data['Document type'] == 'DR', 'Object'].drop_duplicates().to_list() if not project[-3].isnumeric()]

In [16]:
project_lists_alloc

['DO/2021-POOL003',
 'DO/2016-GVDA001',
 'DO/2018-WOMC001',
 'DO/2021-ATPL001',
 'DO/2020-SFSS001',
 'DO/2024-IITBHF0-030',
 'DO/2024-IITBHF0-031',
 'DO/2024-IITBHF0-032',
 'DO/2024-IITBHF0-033',
 'DO/2024-IITBHF0-034',
 'DO/2024-IITBHF0-035',
 'DO/2022-CIIB005',
 'DO/2022-CIIB006',
 'DO/2022-CIIB007',
 'DO/2021-DCIL002-001',
 'DO/2024-RHUB003']

In [17]:
final_data.loc[
    (final_data['Document type'] == 'DR') &
    (final_data['Object'] == 'DO/2024-IITBHF0-030')
]

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category
99585,1.002053e+09,2.406001e+09,2024-04-17,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-8338300.0,-8338300.0,-8338300.0,None,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256,grant_#_256,NaN
99587,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-12507450.0,-12507450.0,-12507450.0,None,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256,grant_#_256,NaN
99588,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-3126862.0,-3126862.0,-3126862.0,None,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256,grant_#_256,NaN
99589,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-3126858.0,-3126858.0,-3126858.0,None,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256,grant_#_256,NaN
99590,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-303931.0,-303931.0,-303931.0,None,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256,grant_#_256,NaN
99591,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,303931.0,303931.0,303931.0,None,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256,grant_#_256,NaN
99592,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-1919357.0,-1919357.0,-1919357.0,None,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256,grant_#_256,NaN
99593,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-216796.0,-216796.0,-216796.0,None,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256,grant_#_256,NaN
99594,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-2584869.0,-2584869.0,-2584869.0,None,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256,grant_#_256,NaN
99595,1.002060e+09,2.406001e+09,2024-04-18,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-1021024.0,-1021024.0,-1021024.0,None,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256,grant_#_256,NaN


In [18]:
final_data.loc[
    (final_data['Document type'] == 'DR') &
    (final_data['Object'] == 'DO/2024-IITBHF0-030'),
    'Val/COArea Crcy'
].sum()

np.float64(-33396326.1)

In [19]:
final_data[
    (final_data['Document type'] == 'SB') &
    (final_data['Object'] == 'DO/2024-IITBHF0-030')
]

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category
99586,1.002053e+09,2.426000e+09,2024-04-17,2024-04-16,SB,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,8338300.0,8338300.0,8338300.0,None,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256,grant_#_256,NaN
99600,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,25058026.1,25058026.1,25058026.1,None,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256,grant_#_256,NaN


In [20]:
final_data.loc[
    (final_data['Document type'] == 'SB') &
    (final_data['Object'] == 'DO/2024-IITBHF0-030'),
    'Val/COArea Crcy'
].sum()

np.float64(33396326.1)

In [21]:
final_data.loc[
    (final_data['Object'] == 'DO/2024-IITBHF0-030'),
    'Ref. document number'
].drop_duplicates()

99585    2.406001e+09
99586    2.426000e+09
99587    2.406001e+09
99588    2.406001e+09
99589    2.406001e+09
99590    2.406001e+09
99591    2.406001e+09
99592    2.406001e+09
99593    2.406001e+09
99594    2.406001e+09
99595    2.406001e+09
99596    2.406001e+09
99597    2.406001e+09
99598    2.406001e+09
99599    2.406001e+09
99600    2.426000e+09
Name: Ref. document number, dtype: float64

In [22]:
final_data[
    final_data['Ref. document number'].isin(final_data.loc[
    (final_data['Object'] == 'DO/2024-IITBHF0-030'),
    'Ref. document number'
].drop_duplicates())
]

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category
8750,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2007-YFA,DO/2007-YFAREV,2024.0,550511.0,Rev-IITBHF Donation,...,-216796.0,-216796.0,-216796.0,None,DO/2007-YFA,CLASS OF'82 LEGACY PROJECT JOINING BONUS,DO/2007-YFAREV,CLASS OF'82 LEGACY PROJECT -REV,class_of'82_legacy_project_-rev,NaN
9565,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2008-COSP,DO/2008-COSPREV,2024.0,550511.0,Rev-IITBHF Donation,...,-2584869.0,-2584869.0,-2584869.0,None,DO/2008-COSP,Class of 1970 Legacy Project,DO/2008-COSPREV,Class of 1970 Legacy Project-REV,class_of_1970_legacy_project-rev,NaN
9566,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2008-COSP,DO/2008-COSPREV,2024.0,550511.0,Rev-IITBHF Donation,...,2584869.0,2584869.0,2584869.0,None,DO/2008-COSP,Class of 1970 Legacy Project,DO/2008-COSPREV,Class of 1970 Legacy Project-REV,class_of_1970_legacy_project-rev,NaN
14194,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2012-DCAD,DO/2012-DCADREV,2024.0,550511.0,Rev-IITBHF Donation,...,-9573.0,-9573.0,-9573.0,None,DO/2012-DCAD,DHAMDHERE CHAIR AWARD,DO/2012-DCADREV,DHAMDHERE CHAIR AWARD,dhamdhere_chair_award,NaN
14195,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2012-DCAD,DO/2012-DCADREV,2024.0,550511.0,Rev-IITBHF Donation,...,-82533.0,-82533.0,-82533.0,None,DO/2012-DCAD,DHAMDHERE CHAIR AWARD,DO/2012-DCADREV,DHAMDHERE CHAIR AWARD,dhamdhere_chair_award,NaN
55338,1.002053e+09,2.426000e+09,2024-04-17,2024-04-16,SB,DO/2015-WEL,DO/2015-WELREV,2024.0,550511.0,Rev-IITBHF Donation,...,-8338300.0,-8338300.0,-8338300.0,None,DO/2015-WEL,WADHAWANI ELECTRONICS LAB,DO/2015-WELREV,WADHAWANI ELECTRONICS LAB-REV,wadhawani_electronics_lab-rev,NaN
89088,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2019-LP94,DO/2019-LP94REV,2024.0,550511.0,Rev-IITBHF Donation,...,-1919357.0,-1919357.0,-1919357.0,None,DO/2019-LP94,CLASS OF 1994 LEGACY PROJECT,DO/2019-LP94REV,CLASS OF 1994 LEGACY PROJECT Receipt,class_of_1994_legacy_project_receipt,NaN
92064,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2019-SMTA,DO/2019-SMTAREV,2024.0,550511.0,Rev-IITBHF Donation,...,-254247.1,-254247.1,-254247.1,None,DO/2019-SMTA,Teach. Award Memory of Prof Sahasrabudhe,DO/2019-SMTAREV,Teach. Award Memory of Prof Sahasrabudhe,teach._award_memory_of_prof_sahasrabudhe,NaN
99585,1.002053e+09,2.406001e+09,2024-04-17,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-8338300.0,-8338300.0,-8338300.0,None,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256,grant_#_256,NaN
99586,1.002053e+09,2.426000e+09,2024-04-17,2024-04-16,SB,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,8338300.0,8338300.0,8338300.0,None,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256,grant_#_256,NaN


In [23]:
final_data.loc[
    final_data['Ref. document number'].isin(final_data.loc[
    (final_data['Object'] == 'DO/2024-IITBHF0-030'),
    'Ref. document number'
].drop_duplicates()),
    'Val/COArea Crcy'
].sum()

np.float64(-33396326.1)

In [24]:
final_data.loc[
    (
        final_data['Ref. document number'].isin(final_data.loc[
            (final_data['Object'] == 'DO/2024-IITBHF0-030'),
            'Ref. document number'
        ].drop_duplicates())
    )
]

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category
8750,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2007-YFA,DO/2007-YFAREV,2024.0,550511.0,Rev-IITBHF Donation,...,-216796.0,-216796.0,-216796.0,None,DO/2007-YFA,CLASS OF'82 LEGACY PROJECT JOINING BONUS,DO/2007-YFAREV,CLASS OF'82 LEGACY PROJECT -REV,class_of'82_legacy_project_-rev,NaN
9565,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2008-COSP,DO/2008-COSPREV,2024.0,550511.0,Rev-IITBHF Donation,...,-2584869.0,-2584869.0,-2584869.0,None,DO/2008-COSP,Class of 1970 Legacy Project,DO/2008-COSPREV,Class of 1970 Legacy Project-REV,class_of_1970_legacy_project-rev,NaN
9566,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2008-COSP,DO/2008-COSPREV,2024.0,550511.0,Rev-IITBHF Donation,...,2584869.0,2584869.0,2584869.0,None,DO/2008-COSP,Class of 1970 Legacy Project,DO/2008-COSPREV,Class of 1970 Legacy Project-REV,class_of_1970_legacy_project-rev,NaN
14194,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2012-DCAD,DO/2012-DCADREV,2024.0,550511.0,Rev-IITBHF Donation,...,-9573.0,-9573.0,-9573.0,None,DO/2012-DCAD,DHAMDHERE CHAIR AWARD,DO/2012-DCADREV,DHAMDHERE CHAIR AWARD,dhamdhere_chair_award,NaN
14195,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2012-DCAD,DO/2012-DCADREV,2024.0,550511.0,Rev-IITBHF Donation,...,-82533.0,-82533.0,-82533.0,None,DO/2012-DCAD,DHAMDHERE CHAIR AWARD,DO/2012-DCADREV,DHAMDHERE CHAIR AWARD,dhamdhere_chair_award,NaN
55338,1.002053e+09,2.426000e+09,2024-04-17,2024-04-16,SB,DO/2015-WEL,DO/2015-WELREV,2024.0,550511.0,Rev-IITBHF Donation,...,-8338300.0,-8338300.0,-8338300.0,None,DO/2015-WEL,WADHAWANI ELECTRONICS LAB,DO/2015-WELREV,WADHAWANI ELECTRONICS LAB-REV,wadhawani_electronics_lab-rev,NaN
89088,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2019-LP94,DO/2019-LP94REV,2024.0,550511.0,Rev-IITBHF Donation,...,-1919357.0,-1919357.0,-1919357.0,None,DO/2019-LP94,CLASS OF 1994 LEGACY PROJECT,DO/2019-LP94REV,CLASS OF 1994 LEGACY PROJECT Receipt,class_of_1994_legacy_project_receipt,NaN
92064,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2019-SMTA,DO/2019-SMTAREV,2024.0,550511.0,Rev-IITBHF Donation,...,-254247.1,-254247.1,-254247.1,None,DO/2019-SMTA,Teach. Award Memory of Prof Sahasrabudhe,DO/2019-SMTAREV,Teach. Award Memory of Prof Sahasrabudhe,teach._award_memory_of_prof_sahasrabudhe,NaN
99585,1.002053e+09,2.406001e+09,2024-04-17,2024-04-16,DR,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,-8338300.0,-8338300.0,-8338300.0,None,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256,grant_#_256,NaN
99586,1.002053e+09,2.426000e+09,2024-04-17,2024-04-16,SB,DO/2020-IITBHF0,DO/2024-IITBHF0-030,2024.0,550511.0,Rev-IITBHF Donation,...,8338300.0,8338300.0,8338300.0,None,DO/2020-IITBHF0,IIT BHF GRANTS,DO/2024-IITBHF0-030,Grant # 256,grant_#_256,NaN


In [25]:
final_data.loc[
    (
        final_data['Ref. document number'].isin(final_data.loc[
            (final_data['Object'] == 'DO/2024-IITBHF0-030'),
            'Ref. document number'
        ].drop_duplicates())
    ) &
    (~final_data['Object'].str[-3:].str.isnumeric()) &
    (final_data['Dr/Cr indicator'] == 'C')
]

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category
8750,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2007-YFA,DO/2007-YFAREV,2024.0,550511.0,Rev-IITBHF Donation,...,-216796.0,-216796.0,-216796.0,None,DO/2007-YFA,CLASS OF'82 LEGACY PROJECT JOINING BONUS,DO/2007-YFAREV,CLASS OF'82 LEGACY PROJECT -REV,class_of'82_legacy_project_-rev,NaN
9565,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2008-COSP,DO/2008-COSPREV,2024.0,550511.0,Rev-IITBHF Donation,...,-2584869.0,-2584869.0,-2584869.0,None,DO/2008-COSP,Class of 1970 Legacy Project,DO/2008-COSPREV,Class of 1970 Legacy Project-REV,class_of_1970_legacy_project-rev,NaN
14194,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2012-DCAD,DO/2012-DCADREV,2024.0,550511.0,Rev-IITBHF Donation,...,-9573.0,-9573.0,-9573.0,None,DO/2012-DCAD,DHAMDHERE CHAIR AWARD,DO/2012-DCADREV,DHAMDHERE CHAIR AWARD,dhamdhere_chair_award,NaN
14195,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2012-DCAD,DO/2012-DCADREV,2024.0,550511.0,Rev-IITBHF Donation,...,-82533.0,-82533.0,-82533.0,None,DO/2012-DCAD,DHAMDHERE CHAIR AWARD,DO/2012-DCADREV,DHAMDHERE CHAIR AWARD,dhamdhere_chair_award,NaN
55338,1.002053e+09,2.426000e+09,2024-04-17,2024-04-16,SB,DO/2015-WEL,DO/2015-WELREV,2024.0,550511.0,Rev-IITBHF Donation,...,-8338300.0,-8338300.0,-8338300.0,None,DO/2015-WEL,WADHAWANI ELECTRONICS LAB,DO/2015-WELREV,WADHAWANI ELECTRONICS LAB-REV,wadhawani_electronics_lab-rev,NaN
89088,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2019-LP94,DO/2019-LP94REV,2024.0,550511.0,Rev-IITBHF Donation,...,-1919357.0,-1919357.0,-1919357.0,None,DO/2019-LP94,CLASS OF 1994 LEGACY PROJECT,DO/2019-LP94REV,CLASS OF 1994 LEGACY PROJECT Receipt,class_of_1994_legacy_project_receipt,NaN
92064,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2019-SMTA,DO/2019-SMTAREV,2024.0,550511.0,Rev-IITBHF Donation,...,-254247.1,-254247.1,-254247.1,None,DO/2019-SMTA,Teach. Award Memory of Prof Sahasrabudhe,DO/2019-SMTAREV,Teach. Award Memory of Prof Sahasrabudhe,teach._award_memory_of_prof_sahasrabudhe,NaN
102941,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2021-CMIN,DO/2021-CMINREV,2024.0,550511.0,Rev-IITBHF Donation,...,-12507450.0,-12507450.0,-12507450.0,None,DO/2021-CMIN,CENTRE FOR MACHINE INTELLIGENCE AND DATA,DO/2021-CMINREV,CMINDS-REVENUE,cminds-revenue,NaN
122832,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2022-IBRT,DO/2022-IBRTREV,2024.0,550511.0,Rev-IITBHF Donation,...,-1021024.0,-1021024.0,-1021024.0,None,DO/2022-IBRT,IIT Bombay Racing Team,DO/2022-IBRTREV,IIT Bombay Racing Team - REV,iit_bombay_racing_team_-_rev,NaN
132350,1.002064e+09,2.426000e+09,2024-04-23,2024-04-16,SB,DO/2023-TR70,DO/2023-TR70REV,2024.0,550511.0,Rev-IITBHF Donation,...,-2584869.0,-2584869.0,-2584869.0,None,DO/2023-TR70,Class of 1970 Legacy Project (TRA),DO/2023-TR70REV,Class of 1970 Legacy Project (TRA)-REV,class_of_1970_legacy_project_(tra)-rev,NaN


In [26]:
final_data.loc[
    (
        final_data['Ref. document number'].isin(final_data.loc[
            (final_data['Object'] == 'DO/2024-IITBHF0-030'),
            'Ref. document number'
        ].drop_duplicates())
    ) &
    (~final_data['Object'].str[-3:].str.isnumeric()) &
    (final_data['Dr/Cr indicator'] == 'C'),
    'Val/COArea Crcy'
].sum()

np.float64(-35981195.1)

In [27]:
final_data[
    final_data['Ref. document number'].isin(['2426000165.0', '2426000394.0'])
]

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category


In [28]:
final_data.loc[
    (final_data['Ref. document number'].isin(['2426000165.0', '2426000394.0'])) &
    (final_data['Dr/Cr indicator'] == 'C')
]

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category


In [29]:
# final_data.loc[
#     (final_data['Ref. document number'].isin(['2426000165.0', '2426000394.0'])) &
#     (final_data['Val/COArea Crcy'] < 0)
#     # (~final_data['Object'].str[-3:].str.isnumeric()),
#     ,'Val/COArea Crcy'
# ].sum()

final_data.loc[
    (final_data['Ref. document number'].isin(['2426000165.0', '2426000394.0'])) &
    (final_data['Dr/Cr indicator'] == 'C'),
    'Val/COArea Crcy'
].sum()

np.float64(0.0)

In [30]:
final_data.loc[
    (final_data['Document type'] == 'DR') &
    (final_data['Object'].isin(project_lists_alloc)),
    'Val/COArea Crcy'
].sum()

np.float64(-422273308.68)

In [31]:
project_lists_non_alloc

['DO/2001-IDPFREV',
 'DO/2003-ARSEREV',
 'DO/2003-CSEDREV',
 'DO/2003-EEDPREV',
 'DO/2007-YFAREV',
 'DO/2007-SSDFREV',
 'DO/2009-HATSREV',
 'DO/2014-PDMAREV',
 'DO/2015-BSBEREV',
 'DO/2015-HPCLREV',
 'DO/2016-RKMAREV',
 'DO/2017-FIAPREV',
 'DO/2017-LP92REV',
 'DO/2018-CL98REV',
 'DO/2018-FAPRREV',
 'DO/2019-CECRREV',
 'DO/2019-CSRPREV',
 'DO/2019-INNFREV',
 'DO/2019-LP94REV',
 'DO/2020-C95LREV',
 'DO/2020-CWAIREV',
 'DO/2020-SFSSREV',
 'DO/2020-UNDNREV',
 'DO/2021-CCSEREV',
 'DO/2021-COGEREV',
 'DO/2021-CTINREV',
 'DO/2021-GFLLREV',
 'DO/2021-GJ71REV',
 'DO/2021-HSPEREV',
 'DO/2021-ILSBREV',
 'DO/2021-LP96REV',
 'DO/2021-RDPJREV',
 'DO/2021-SITPREV',
 'DO/2021-TCEPREV',
 'DO/2021-WSSSREV',
 'DO/2022-CDSLREV',
 'DO/2022-CSBGREV',
 'DO/2022-IITMREV',
 'DO/2022-LP97REV',
 'DO/2022-SBSTREV',
 'DO/2022-SSIPREV',
 'DO/2022-STEMREV',
 'DO/2023-C73AREV',
 'DO/2023-CTDLREV',
 'DO/2023-DEPTREV',
 'DO/2023-HSBCREV',
 'DO/2023-KCORREV',
 'DO/2023-LP83REV',
 'DO/2023-MDSLREV',
 'DO/2023-RANGREV',
 

In [32]:
final_data.loc[
    (final_data['Document type'] == 'DR') &
    (final_data['Object'].isin(project_lists_non_alloc)),
    'Val/COArea Crcy'
].sum()

np.float64(-730097428.86)

In [33]:
project_lists_non_alloc

['DO/2001-IDPFREV',
 'DO/2003-ARSEREV',
 'DO/2003-CSEDREV',
 'DO/2003-EEDPREV',
 'DO/2007-YFAREV',
 'DO/2007-SSDFREV',
 'DO/2009-HATSREV',
 'DO/2014-PDMAREV',
 'DO/2015-BSBEREV',
 'DO/2015-HPCLREV',
 'DO/2016-RKMAREV',
 'DO/2017-FIAPREV',
 'DO/2017-LP92REV',
 'DO/2018-CL98REV',
 'DO/2018-FAPRREV',
 'DO/2019-CECRREV',
 'DO/2019-CSRPREV',
 'DO/2019-INNFREV',
 'DO/2019-LP94REV',
 'DO/2020-C95LREV',
 'DO/2020-CWAIREV',
 'DO/2020-SFSSREV',
 'DO/2020-UNDNREV',
 'DO/2021-CCSEREV',
 'DO/2021-COGEREV',
 'DO/2021-CTINREV',
 'DO/2021-GFLLREV',
 'DO/2021-GJ71REV',
 'DO/2021-HSPEREV',
 'DO/2021-ILSBREV',
 'DO/2021-LP96REV',
 'DO/2021-RDPJREV',
 'DO/2021-SITPREV',
 'DO/2021-TCEPREV',
 'DO/2021-WSSSREV',
 'DO/2022-CDSLREV',
 'DO/2022-CSBGREV',
 'DO/2022-IITMREV',
 'DO/2022-LP97REV',
 'DO/2022-SBSTREV',
 'DO/2022-SSIPREV',
 'DO/2022-STEMREV',
 'DO/2023-C73AREV',
 'DO/2023-CTDLREV',
 'DO/2023-DEPTREV',
 'DO/2023-HSBCREV',
 'DO/2023-KCORREV',
 'DO/2023-LP83REV',
 'DO/2023-MDSLREV',
 'DO/2023-RANGREV',
 

In [34]:
project_wise_data = pd.DataFrame()

for index, row in final_data[final_data['Document type'] == 'DR'].iterrows():
    sb_entry = sb_entries[
        (sb_entries['Object'] == row['Object']) &
        (not row['Object'][-3].isnumeric())
    ]

    transactions = pd.DataFrame()

    for index1, row1 in sb_entry.iterrows():
        df = final_data[
            (final_data['Ref. document number'] == row1['Ref. document number']) &
            (final_data['Val/COArea Crcy'] < 0)
        ]
        transactions = pd.concat([transactions, df])

    if sb_entry.shape[0] > 0:
        # print(row['Object'])
        # break
        project_wise_data = pd.concat([project_wise_data, transactions])

    else:
        df = final_data[(final_data['Document type'] == 'DR') & (final_data['Object'] == row['Object'])]
        project_wise_data = pd.concat([project_wise_data, df])

In [35]:
project_wise_data

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category
1227,1.002047e+09,2.406000e+09,2024-04-15,2024-04-01,DR,DO/2000-POOL,DO/2021-POOL003,2024.0,550508.0,Rev-Domestic (Indian fund),...,-501.0,-501.0,-501.0,None,DO/2000-POOL,Donation Pool Account,DO/2021-POOL003,DOMESTIC SCHOLARSHIP,domestic_scholarship,NaN
1228,1.002047e+09,2.406000e+09,2024-04-15,2024-04-02,DR,DO/2000-POOL,DO/2021-POOL003,2024.0,550508.0,Rev-Domestic (Indian fund),...,-501.0,-501.0,-501.0,None,DO/2000-POOL,Donation Pool Account,DO/2021-POOL003,DOMESTIC SCHOLARSHIP,domestic_scholarship,NaN
1229,1.002047e+09,2.406000e+09,2024-04-15,2024-04-03,DR,DO/2000-POOL,DO/2021-POOL003,2024.0,550508.0,Rev-Domestic (Indian fund),...,-25.0,-25.0,-25.0,None,DO/2000-POOL,Donation Pool Account,DO/2021-POOL003,DOMESTIC SCHOLARSHIP,domestic_scholarship,NaN
1230,1.002047e+09,2.406000e+09,2024-04-15,2024-04-05,DR,DO/2000-POOL,DO/2021-POOL003,2024.0,550508.0,Rev-Domestic (Indian fund),...,-501.0,-501.0,-501.0,None,DO/2000-POOL,Donation Pool Account,DO/2021-POOL003,DOMESTIC SCHOLARSHIP,domestic_scholarship,NaN
1231,1.002047e+09,2.406000e+09,2024-04-15,2024-04-06,DR,DO/2000-POOL,DO/2021-POOL003,2024.0,550508.0,Rev-Domestic (Indian fund),...,-501.0,-501.0,-501.0,None,DO/2000-POOL,Donation Pool Account,DO/2021-POOL003,DOMESTIC SCHOLARSHIP,domestic_scholarship,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133338,1.002249e+09,2.406012e+09,2024-11-22,2024-11-15,DR,DO/2000-LP74,DO/2000-LP74REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-100000.0,-100000.0,-100000.0,None,DO/2000-LP74,CLASS OF'74,DO/2000-LP74REV,CLASS OF 1974 LEGACY PROJECT,class_of_1974_legacy_project,Others
133339,1.002253e+09,2.406012e+09,2024-11-28,2024-11-21,DR,DO/2000-LP74,DO/2000-LP74REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-200000.0,-200000.0,-200000.0,None,DO/2000-LP74,CLASS OF'74,DO/2000-LP74REV,CLASS OF 1974 LEGACY PROJECT,class_of_1974_legacy_project,Others
133340,1.002253e+09,2.406012e+09,2024-11-29,2024-11-24,DR,DO/2000-LP74,DO/2000-LP74REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-200000.0,-200000.0,-200000.0,None,DO/2000-LP74,CLASS OF'74,DO/2000-LP74REV,CLASS OF 1974 LEGACY PROJECT,class_of_1974_legacy_project,Others
133341,1.002254e+09,2.406012e+09,2024-11-29,2024-11-25,DR,DO/2000-LP74,DO/2000-LP74REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-12000.0,-12000.0,-12000.0,None,DO/2000-LP74,CLASS OF'74,DO/2000-LP74REV,CLASS OF 1974 LEGACY PROJECT,class_of_1974_legacy_project,Others


In [36]:
project_wise_data['Val/COArea Crcy'].sum()

np.float64(-49257118134.48)

In [37]:
transactions

""


In [38]:
sb_entry

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category


In [39]:
sb_entry.loc[sb_entry['Val/COArea Crcy'] < 0, 'Val/COArea Crcy'].sum()

np.float64(0.0)

In [40]:
final_data.loc[
    (
        (final_data['Object'] == 'DO/2001-IDPFREV') &
        (final_data['Document type'] == 'DR')
    )
]

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category
2425,1.002047e+09,2.406000e+09,2024-04-15,2024-04-02,DR,DO/2001-IDPF,DO/2001-IDPFREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-500.0,-500.0,-500.0,None,DO/2001-IDPF,INSTITUTE DEVELOPMENT FUND,DO/2001-IDPFREV,INSTITUTE DEVELOPMENT FUND,institute_development_fund,Strategic Needs
2426,1.002048e+09,2.406000e+09,2024-04-15,2024-04-05,DR,DO/2001-IDPF,DO/2001-IDPFREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-100.0,-100.0,-100.0,None,DO/2001-IDPF,INSTITUTE DEVELOPMENT FUND,DO/2001-IDPFREV,INSTITUTE DEVELOPMENT FUND,institute_development_fund,Strategic Needs
2427,1.002048e+09,2.406000e+09,2024-04-15,2024-04-10,DR,DO/2001-IDPF,DO/2001-IDPFREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-17.0,-17.0,-17.0,None,DO/2001-IDPF,INSTITUTE DEVELOPMENT FUND,DO/2001-IDPFREV,INSTITUTE DEVELOPMENT FUND,institute_development_fund,Strategic Needs
2428,1.002051e+09,2.406001e+09,2024-04-17,2024-04-16,DR,DO/2001-IDPF,DO/2001-IDPFREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-1008.0,-1008.0,-1008.0,None,DO/2001-IDPF,INSTITUTE DEVELOPMENT FUND,DO/2001-IDPFREV,INSTITUTE DEVELOPMENT FUND,institute_development_fund,Strategic Needs
2429,1.002063e+09,2.406001e+09,2024-04-22,2024-04-19,DR,DO/2001-IDPF,DO/2001-IDPFREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-2000.0,-2000.0,-2000.0,None,DO/2001-IDPF,INSTITUTE DEVELOPMENT FUND,DO/2001-IDPFREV,INSTITUTE DEVELOPMENT FUND,institute_development_fund,Strategic Needs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2504,1.002243e+09,2.406012e+09,2024-11-18,2024-11-16,DR,DO/2001-IDPF,DO/2001-IDPFREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-1008.0,-1008.0,-1008.0,None,DO/2001-IDPF,INSTITUTE DEVELOPMENT FUND,DO/2001-IDPFREV,INSTITUTE DEVELOPMENT FUND,institute_development_fund,Strategic Needs
2505,1.002248e+09,2.406012e+09,2024-11-21,2024-11-19,DR,DO/2001-IDPF,DO/2001-IDPFREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-2000.0,-2000.0,-2000.0,None,DO/2001-IDPF,INSTITUTE DEVELOPMENT FUND,DO/2001-IDPFREV,INSTITUTE DEVELOPMENT FUND,institute_development_fund,Strategic Needs
2506,1.002255e+09,2.406012e+09,2024-12-02,2024-11-29,DR,DO/2001-IDPF,DO/2001-IDPFREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-20000.0,-20000.0,-20000.0,None,DO/2001-IDPF,INSTITUTE DEVELOPMENT FUND,DO/2001-IDPFREV,INSTITUTE DEVELOPMENT FUND,institute_development_fund,Strategic Needs
2507,1.002255e+09,2.406012e+09,2024-12-02,2024-12-01,DR,DO/2001-IDPF,DO/2001-IDPFREV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-1001.0,-1001.0,-1001.0,None,DO/2001-IDPF,INSTITUTE DEVELOPMENT FUND,DO/2001-IDPFREV,INSTITUTE DEVELOPMENT FUND,institute_development_fund,Strategic Needs


In [41]:
final_data.loc[
    (
        (final_data['Object'] == 'DO/2001-IDPFREV') &
        (final_data['Document type'] == 'DR')
    ),
    'Val/COArea Crcy'].sum()

np.float64(-623480.0)

In [42]:
final_data[final_data['Ref. document number'] == '2426008574.0']

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category


In [43]:
final_data[final_data['Ref. document number'] == '2426003398.0']

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category


In [44]:
final_data

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category
551,1.002162e+09,2.426005e+09,2024-08-23,2024-08-23,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,510508.0,EXP-SCHOLARSHIP,...,38348.88,38348.88,38348.88,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
552,1.002162e+09,2.426005e+09,2024-08-23,2024-08-23,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,510508.0,EXP-SCHOLARSHIP,...,36109.74,36109.74,36109.74,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
555,1.002165e+09,2.426005e+09,2024-08-28,2024-08-27,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,510508.0,EXP-SCHOLARSHIP,...,38348.88,38348.88,38348.88,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
556,1.002165e+09,2.426005e+09,2024-08-28,2024-08-27,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,510508.0,EXP-SCHOLARSHIP,...,36109.74,36109.74,36109.74,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
912,1.002200e+09,2.426007e+09,2024-10-14,2024-10-01,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,550511.0,Rev-IITBHF Donation,...,-32196387.45,-32196387.45,-32196387.45,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133338,1.002249e+09,2.406012e+09,2024-11-22,2024-11-15,DR,DO/2000-LP74,DO/2000-LP74REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-100000.00,-100000.00,-100000.00,None,DO/2000-LP74,CLASS OF'74,DO/2000-LP74REV,CLASS OF 1974 LEGACY PROJECT,class_of_1974_legacy_project,Others
133339,1.002253e+09,2.406012e+09,2024-11-28,2024-11-21,DR,DO/2000-LP74,DO/2000-LP74REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-200000.00,-200000.00,-200000.00,None,DO/2000-LP74,CLASS OF'74,DO/2000-LP74REV,CLASS OF 1974 LEGACY PROJECT,class_of_1974_legacy_project,Others
133340,1.002253e+09,2.406012e+09,2024-11-29,2024-11-24,DR,DO/2000-LP74,DO/2000-LP74REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-200000.00,-200000.00,-200000.00,None,DO/2000-LP74,CLASS OF'74,DO/2000-LP74REV,CLASS OF 1974 LEGACY PROJECT,class_of_1974_legacy_project,Others
133341,1.002254e+09,2.406012e+09,2024-11-29,2024-11-25,DR,DO/2000-LP74,DO/2000-LP74REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-12000.00,-12000.00,-12000.00,None,DO/2000-LP74,CLASS OF'74,DO/2000-LP74REV,CLASS OF 1974 LEGACY PROJECT,class_of_1974_legacy_project,Others


## Check HF Donations  distribution

In [45]:
final_data = final_data.reset_index(drop=True).copy()

In [46]:
final_data['Document Number'] = final_data['Document Number'].fillna('').astype(str)

In [47]:
final_data['Ref. document number'] = final_data['Ref. document number'].fillna('').astype(str)

In [48]:
final_data.loc[
    (final_data['Document type'] == 'DR') &
    (final_data['Cost Element'] == 550511),
    'Val/COArea Crcy'
].sum()

np.float64(-414479474.68)

In [49]:
hf_objects = final_data.loc[
    (final_data['Document type'] == 'DR') &
    (final_data['Cost Element'] == 550511),
    'Object'
].drop_duplicates().tolist()

In [50]:
hf_objects

['DO/2024-IITBHF0-030',
 'DO/2024-IITBHF0-031',
 'DO/2024-IITBHF0-032',
 'DO/2024-IITBHF0-033',
 'DO/2024-IITBHF0-034',
 'DO/2024-IITBHF0-035']

In [51]:
final_data.loc[
    (final_data['Object'].isin(hf_objects)) &
    (final_data['Document type'] == 'SB'),
    'Val/COArea Crcy'
].sum()

np.float64(414479474.67999995)

In [52]:
hf_references = final_data.loc[
    (final_data['Object'].isin(hf_objects)) &
    (final_data['Document type'] == 'SB'),
    'Ref. document number'
].drop_duplicates().dropna().tolist()

In [53]:
hf_references

['2426000165.0',
 '2426000394.0',
 '2426001585.0',
 '2426001601.0',
 '2426001935.0',
 '2426003318.0',
 '2426003335.0',
 '2426003577.0',
 '2426003580.0',
 '2426003583.0',
 '2426007241.0',
 '2426007281.0']

In [54]:
final_data.loc[
    (final_data['Ref. document number'].isin(hf_references)),
    :
]

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category
4,1002199703.0,2426007241.0,2024-10-14,2024-10-01,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,550511.0,Rev-IITBHF Donation,...,-32196387.45,-32196387.45,-32196387.45,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
5,1002199703.0,2426007241.0,2024-10-14,2024-10-01,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,550511.0,Rev-IITBHF Donation,...,32196387.45,32196387.45,32196387.45,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
258,1002199703.0,2426007241.0,2024-10-14,2024-10-01,SB,DO/2000-POOL,DO/2024-POOL002-004,2024.0,550511.0,Rev-IITBHF Donation,...,-32196387.45,-32196387.45,-32196387.45,None,DO/2000-POOL,Donation Pool Account,DO/2024-POOL002-004,IITBHF GRANT #259,iitbhf_grant_#259,NaN
614,1002123887.0,2426003335.0,2024-07-12,2024-07-05,SB,DO/2002-VVSV,DO/2002-VVSVREV,2024.0,550511.0,Rev-IITBHF Donation,...,-6250275.00,-6250275.00,-6250275.00,None,DO/2002-VVSV,VIJAY VASHEE FOR VIJAY & SITA VASHEE CHA,DO/2002-VVSVREV,VIJAY VASHEE FOR VIJAY & SITA VASHEE CHA,vijay_vashee_for_vijay_&_sita_vashee_cha,NaN
615,1002200540.0,2426007281.0,2024-10-14,2024-10-14,SB,DO/2002-VVSV,DO/2002-VVSVREV,2024.0,550511.0,Rev-IITBHF Donation,...,-16733800.00,-16733800.00,-16733800.00,None,DO/2002-VVSV,VIJAY VASHEE FOR VIJAY & SITA VASHEE CHA,DO/2002-VVSVREV,VIJAY VASHEE FOR VIJAY & SITA VASHEE CHA,vijay_vashee_for_vijay_&_sita_vashee_cha,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14847,1002063982.0,2426000394.0,2024-04-23,2024-04-16,SB,DO/2024-VBAR,DO/2024-VBARREV,2024.0,550511.0,Rev-IITBHF Donation,...,-208457.00,-208457.00,-208457.00,None,DO/2024-VBAR,Dr. Varun Bhalerao's Astronomy Related A,DO/2024-VBARREV,Dr. Varun Bhalerao's Astronomy Rela-REV,dr._varun_bhalerao's_astronomy_rela-rev,NaN
14853,1002063982.0,2426000394.0,2024-04-23,2024-04-16,SB,DO/2024-VCCF,DO/2024-VCCFREV,2024.0,550511.0,Rev-IITBHF Donation,...,-3126858.00,-3126858.00,-3126858.00,None,DO/2024-VCCF,Vijay Vashee Climate Change Fellowship A,DO/2024-VCCFREV,Vijay Vashee Climate Change Fellow-REV,vijay_vashee_climate_change_fellow-rev,NaN
14854,1002123887.0,2426003335.0,2024-07-12,2024-07-05,SB,DO/2024-VCCF,DO/2024-VCCFREV,2024.0,550511.0,Rev-IITBHF Donation,...,-1250040.00,-1250040.00,-1250040.00,None,DO/2024-VCCF,Vijay Vashee Climate Change Fellowship A,DO/2024-VCCFREV,Vijay Vashee Climate Change Fellow-REV,vijay_vashee_climate_change_fellow-rev,NaN
14860,1002063982.0,2426000394.0,2024-04-23,2024-04-16,SB,DO/2024-VCSF,DO/2024-VCSFREV,2024.0,550511.0,Rev-IITBHF Donation,...,-3126862.00,-3126862.00,-3126862.00,None,DO/2024-VCSF,Vijay Vashee Computer Science Fellowship,DO/2024-VCSFREV,Vijay Vashee Computer Science Fell-REV,vijay_vashee_computer_science_fell-rev,NaN


In [55]:
final_data.loc[
    (final_data['Ref. document number'].isin(hf_references)) &
    (~final_data['Object'].str[-3:].str.isnumeric()),
    ['Val/COArea Crcy']
].sum()

Val/COArea Crcy   -3.463082e+08
dtype: float64

In [56]:
final_data.loc[
    (final_data['Document type'] == 'SB') &
    (final_data['Cost Element'] == 550511) &
    (final_data['Dr/Cr indicator'] == 'D'),
    'Val/COArea Crcy'
].sum()

np.float64(1128160636.46)

In [57]:
final_data.loc[
    (final_data['Document type'] == 'SB') &
    (final_data['Cost Element'] == 550511),
    :
]

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category
4,1002199703.0,2426007241.0,2024-10-14,2024-10-01,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,550511.0,Rev-IITBHF Donation,...,-32196387.45,-32196387.45,-32196387.45,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
5,1002199703.0,2426007241.0,2024-10-14,2024-10-01,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,550511.0,Rev-IITBHF Donation,...,32196387.45,32196387.45,32196387.45,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
258,1002199703.0,2426007241.0,2024-10-14,2024-10-01,SB,DO/2000-POOL,DO/2024-POOL002-004,2024.0,550511.0,Rev-IITBHF Donation,...,-32196387.45,-32196387.45,-32196387.45,None,DO/2000-POOL,Donation Pool Account,DO/2024-POOL002-004,IITBHF GRANT #259,iitbhf_grant_#259,NaN
614,1002123887.0,2426003335.0,2024-07-12,2024-07-05,SB,DO/2002-VVSV,DO/2002-VVSVREV,2024.0,550511.0,Rev-IITBHF Donation,...,-6250275.00,-6250275.00,-6250275.00,None,DO/2002-VVSV,VIJAY VASHEE FOR VIJAY & SITA VASHEE CHA,DO/2002-VVSVREV,VIJAY VASHEE FOR VIJAY & SITA VASHEE CHA,vijay_vashee_for_vijay_&_sita_vashee_cha,NaN
615,1002200540.0,2426007281.0,2024-10-14,2024-10-14,SB,DO/2002-VVSV,DO/2002-VVSVREV,2024.0,550511.0,Rev-IITBHF Donation,...,-16733800.00,-16733800.00,-16733800.00,None,DO/2002-VVSV,VIJAY VASHEE FOR VIJAY & SITA VASHEE CHA,DO/2002-VVSVREV,VIJAY VASHEE FOR VIJAY & SITA VASHEE CHA,vijay_vashee_for_vijay_&_sita_vashee_cha,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14847,1002063982.0,2426000394.0,2024-04-23,2024-04-16,SB,DO/2024-VBAR,DO/2024-VBARREV,2024.0,550511.0,Rev-IITBHF Donation,...,-208457.00,-208457.00,-208457.00,None,DO/2024-VBAR,Dr. Varun Bhalerao's Astronomy Related A,DO/2024-VBARREV,Dr. Varun Bhalerao's Astronomy Rela-REV,dr._varun_bhalerao's_astronomy_rela-rev,NaN
14853,1002063982.0,2426000394.0,2024-04-23,2024-04-16,SB,DO/2024-VCCF,DO/2024-VCCFREV,2024.0,550511.0,Rev-IITBHF Donation,...,-3126858.00,-3126858.00,-3126858.00,None,DO/2024-VCCF,Vijay Vashee Climate Change Fellowship A,DO/2024-VCCFREV,Vijay Vashee Climate Change Fellow-REV,vijay_vashee_climate_change_fellow-rev,NaN
14854,1002123887.0,2426003335.0,2024-07-12,2024-07-05,SB,DO/2024-VCCF,DO/2024-VCCFREV,2024.0,550511.0,Rev-IITBHF Donation,...,-1250040.00,-1250040.00,-1250040.00,None,DO/2024-VCCF,Vijay Vashee Climate Change Fellowship A,DO/2024-VCCFREV,Vijay Vashee Climate Change Fellow-REV,vijay_vashee_climate_change_fellow-rev,NaN
14860,1002063982.0,2426000394.0,2024-04-23,2024-04-16,SB,DO/2024-VCSF,DO/2024-VCSFREV,2024.0,550511.0,Rev-IITBHF Donation,...,-3126862.00,-3126862.00,-3126862.00,None,DO/2024-VCSF,Vijay Vashee Computer Science Fellowship,DO/2024-VCSFREV,Vijay Vashee Computer Science Fell-REV,vijay_vashee_computer_science_fell-rev,NaN


In [58]:
final_data

,Document Number,Ref. document number,Posting Date,Document Date,Document type,Project definition,Object,Fiscal Year,Cost Element,Cost element descr.,...,Vbl. value/TranCurr.,Vbl. value/Obj. curr,Var.val.in rep.cur.,Purchase order text,WBS,WBS Details,Sub WBS,SUB WBS Details,Project Id,Category
0,1002161674.0,2426004989.0,2024-08-23,2024-08-23,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,510508.0,EXP-SCHOLARSHIP,...,38348.88,38348.88,38348.88,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
1,1002161674.0,2426004989.0,2024-08-23,2024-08-23,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,510508.0,EXP-SCHOLARSHIP,...,36109.74,36109.74,36109.74,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
2,1002164961.0,2426005166.0,2024-08-28,2024-08-27,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,510508.0,EXP-SCHOLARSHIP,...,38348.88,38348.88,38348.88,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
3,1002164961.0,2426005166.0,2024-08-28,2024-08-27,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,510508.0,EXP-SCHOLARSHIP,...,36109.74,36109.74,36109.74,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
4,1002199703.0,2426007241.0,2024-10-14,2024-10-01,SB,DO/2000-POOL,DO/2000-POOL002,2024.0,550511.0,Rev-IITBHF Donation,...,-32196387.45,-32196387.45,-32196387.45,None,DO/2000-POOL,Donation Pool Account,DO/2000-POOL002,IIT BHF SCHOLARSHIP,iit_bhf_scholarship,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14904,1002249295.0,2406012064.0,2024-11-22,2024-11-15,DR,DO/2000-LP74,DO/2000-LP74REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-100000.00,-100000.00,-100000.00,None,DO/2000-LP74,CLASS OF'74,DO/2000-LP74REV,CLASS OF 1974 LEGACY PROJECT,class_of_1974_legacy_project,Others
14905,1002252504.0,2406012163.0,2024-11-28,2024-11-21,DR,DO/2000-LP74,DO/2000-LP74REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-200000.00,-200000.00,-200000.00,None,DO/2000-LP74,CLASS OF'74,DO/2000-LP74REV,CLASS OF 1974 LEGACY PROJECT,class_of_1974_legacy_project,Others
14906,1002253488.0,2406012186.0,2024-11-29,2024-11-24,DR,DO/2000-LP74,DO/2000-LP74REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-200000.00,-200000.00,-200000.00,None,DO/2000-LP74,CLASS OF'74,DO/2000-LP74REV,CLASS OF 1974 LEGACY PROJECT,class_of_1974_legacy_project,Others
14907,1002253540.0,2406012188.0,2024-11-29,2024-11-25,DR,DO/2000-LP74,DO/2000-LP74REV,2024.0,550508.0,Rev-Domestic (Indian fund),...,-12000.00,-12000.00,-12000.00,None,DO/2000-LP74,CLASS OF'74,DO/2000-LP74REV,CLASS OF 1974 LEGACY PROJECT,class_of_1974_legacy_project,Others
